In [1]:
from flask import Flask, redirect, url_for, request, jsonify
from flask import send_file
import random

app = Flask(__name__)

@app.route('/')
def hello_world():

    a = random.randint(1,30)

    path_to_file = "audio/" + str(a) + ".wav" # ganti dengan text_to_speech_process()
    
    return send_file(
    path_to_file, 
    mimetype="audio/wav", 
    as_attachment=True, 
    download_name="audio.wav")
    return 'Hello, World'

In [2]:
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [21/Oct/2023 16:01:51] "GET / HTTP/1.1" 200 -


In [5]:
app.run(host='localhost', port='5001')

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:5001
Press CTRL+C to quit
127.0.0.1 - - [21/Oct/2023 15:51:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2023 15:51:36] "GET /favicon.ico HTTP/1.1" 404 -
